### **Patrón de diseño de carga de datos**

La terminología utilizada aquí puede ser diferente en sus organizaciones, pero los conceptos serán más o menos los mismos. Hasta ahora en nuestro diseño, tomamos todos los datos que recibimos en el raw container, procesamos todos los datos y reemplazamos todo en nuestro processed container y en el presentation container cada vez. Esta es una arquitectura simple y funciona bien cuando tienes pequeñas cantidades de datos, y esto se llama generalmente un **Full load**. No será adecuado para grandes pipelines de datos, principalmente debido a la cantidad de procesamiento que tendremos que realizar para procesar todos los datos cada vez. Estos pipelines suelen estar diseñados para cargar y procesar sólo los datos que han cambiado entre la ejecución actual y la anterior. Y esto se llama generalmente **Incremental load** (carga incremental). Generalmente, estos dos tipos de cargas están dictados por la cantidad de datos que hemos recibido del sistema fuente, ya sean todos los datos o sólo los cambios. Por ejemplo, si recibimos todos los datos cada vez, es un buen candidato para una carga completa (Full load). Pero si en cambio recibimos sólo los datos que han cambiado desde la última carga, entonces es un candidato para una carga incremental. Pero esto es sólo una orientación, hay técnicas que podemos aplicar para crear un **full load** a partir de un **incremental dataset** y viceversa, pero estas cosas hacen que el procesamiento sea más complejo y no vamos a entrar en ellas en estas lecciones. 

<center><img src="https://i.postimg.cc/3RZWJ7js/db128.png"></center>

Empecemos por entender el **Full Dataset**. En este escenario, el sistema fuente proporciona todos los datos desde el primer día para cada carga. Tomemos como ejemplo nuestro proyecto de Fórmula Uno para entenderlo mejor. El día 1 recibirá los datos de la Race 1. El día 2 no sólo recibirá los datos relacionados con la Race 2, también recibirá los datos del día 1, que son los datos de la Race 1. Del mismo modo, el día 3 recibirá los datos relacionados con la Race 1, Race 2, así como la Race 3 y el día 4 recibirá los datos relacionados con la Race 1, Race 2, Race 3 y Race 4. Y así sucesivamente. Así que tus datos crecen a medida que avanza el día y se hacen más y más grandes. 

<center><img src="https://i.postimg.cc/pV6k2t0h/db129.png"></center>

Veamos ahora el procesamiento que necesitamos para procesar el conjunto completo de datos. La arquitectura que ves aquí te resultará familiar porque es la que acabamos de implementar para nuestro proyecto de Fórmula 1. El primer día, los datos incluyen únicamente información de la "Race 1". El proceso de **Ingestion** sólo crea los datos en el Data lake bajo el contenedor **processed**. Y del mismo modo, el proceso **Transformation** crea esos datos en el Data lake bajo la capa **presentation**. Esos dos contenedores no tenían datos antes de que empezáramos este proceso y ahora tendrán los datos de la Race 1. 

<center><img src="https://i.postimg.cc/prHXxG11/db130.png"></center>

El segundo día, los datos entrantes incluyen información de las Races 1 y 2. Si el proceso de **Ingestion** añade estos datos al Data lake, habrá duplicados. Entonces se hace una sobreescritura (override) que hace que se borren los datos existentes anteriores y se añadan los nuevos datos al Data lake. El proceso **Transformation** hace exactamente lo mismo, borra los datos de la Race 1 que ya están ahí, y luego añade los nuevos datos que hemos recibido, que son los de la Race 1 y 2. 

<center><img src="https://i.postimg.cc/ZRrvryG2/db131.png"></center>

En el día tres, sucede lo mismo. Los datos entrantes incluyen las Races 1, 2 y 3. El proceso de **Ingestion** elimina los datos existentes, que son de las Races 1 y 2 del Data lake, y agrega los datos nuevos tal como se reciben. El proceso de **Transformation** hace exactamente lo mismo y el proceso se repite. Como se puede ver, el problema aquí es que estamos procesando los datos de todas las carreras en cada ejecución, aunque solo los datos del día reciente o la Race reciente han cambiado. Esto agrega una cantidad significativa de procesamiento requerido para la ingestión, así como para la transformación, lo que potencialmente agrega latencia a los datos que se consumen y también se requiere más potencia de procesamiento, lo que resulta potencialmente en costos más altos para su aplicación. Habiendo dicho eso, este modelo ofrece el diseño más simple posible en el que podemos sobrescribir los datos sin preocuparnos por perder nada.

<center><img src="https://i.postimg.cc/SKtjCf1g/db132.png"></center>

Ahora que entendemos el **Full dataset** y el **Full refresh** de los **Full load patterns**, echemos un vistazo al **Incremental dataset**. Como dijimos antes, **Incremental dataset** solo contendrá los cambios del dataset anterior que recibimos. Una vez más, tomemos nuestro dataset de Fórmula Uno como ejemplo para comprenderlo mejor. Similar al **Full dataset** obtendrá los datos de la Race 1 el primer día. El segundo día, a diferencia del **Full dataset**, solo obtendrá los datos de la Race 2. El día tres incluirá los datos de la Race 3 y el día cuatro incluirá los datos de la Race 4, etc.

<center><img src="https://i.postimg.cc/yYdRgWN6/db133.png"></center>

Ahora veamos cómo funcionaría un **Incremental load**. El primer día, no hay cambios entre un **Full load** (carga completa) y un **Incremental load** (carga incremental). Tanto el contenedor de **processed** como el de **presentation** están vacíos al principio, por lo que los datos del primer día se devolverán a ellos.

<center><img src="https://i.postimg.cc/Wb39NNLg/db134.png"></center>

El segundo día será muy diferente. El segundo día sólo se recibirán los datos de la Race 2, como hemos dicho antes, por lo que el proceso de **Ingestion** añadirá (append) los datos de la Race 2 al Data lake. El proceso de **Transformation** ahora tendrá que identificar los cambios de los datos ingeridos y procesar sólo eso y esto resultará en el Data lake para la capa de **presentation** que se añadirá sólo con los datos de la Race 2. Usted podría estar pensando que esto es algo fácil de hacer porque a medida que avanzamos con los días, sólo tenemos que añadir los datos adicionales a los datos a medida que avanzamos. Pero esto no es tan simple como eso, porque tendremos que considerar escenarios de re-ejecución donde tendremos que reemplazar sólo los datos que necesitamos reemplazar en lugar de simplemente sobrescribir todo el Data lake con los nuevos datos que se reciben. Y también en escenarios del mundo real, faltarán datos el primer día, que podrían acabar llegando el segundo día, por ejemplo. Así que tendremos que lidiar con muchos otros escenarios. Así que las cargas incrementales suelen ser complejas de implementar. Así que tenemos que diseñar un sistema inteligente que sólo extraiga los datos necesarios de los Data lakes y los sustituya en casos como los escenarios de re-ejecución.

<center><img src="https://i.postimg.cc/Z5fvTmmB/db135.png"></center>

El día tres sigue un patrón similar al día dos, sólo pasa los datos de la Race 3 al Data lake tanto en el contenedor de **processed** como en el de **presentation**. Así que eso es lo principal a los patrones de diseño que se siguen y una **carga incremental**. 

<center><img src="https://i.postimg.cc/pTKdnbwB/db136.png"></center>

Aparte de estos dos escenarios directos que hemos discutido, hay una serie de **Escenarios híbridos** implementados en proyectos del mundo real. Esto podría ser debido a la naturaleza de los datos que se reciben, y es muy complejo por el tipo y la granularidad de las transformaciones que se están haciendo. Veamos algunos de estos escenarios. 

1. Se puede recibir un Full dataset, pero aún así optar por procesarlos de forma incremental. Esto podría deberse a que queremos obtener algunas ganancias de rendimiento de nuestra transformación y de las capas de ingestión. Esto requeriría otro proceso previo para identificar los cambios. Si tomamos como ejemplo la Fórmula 1, de nuevo, el primer día recibimos los datos de la Carrera 1, el segundo día recibimos los datos de las Carreras 1 y 2. Entonces llegamos a procesar los datos del día dos, podemos introducir ese nuevo proceso del que estamos hablando en un frente para excluir todos los datos relacionados con el día uno, los datos de la Race 1 y tomar únicamente los datos de la Race 2. Así que vamos a terminar únicamente con los datos de la Race 2 y podemos procesar incrementalmente esto y mejorar el rendimiento de cara a la transformación. 

<center><img src="https://i.postimg.cc/5239JSws/db137.png"></center>

2. Pueden recibir un incremental dataset, pero procesarlo en su totalidad. Esto es ligeramente opuesto a la primera opción. Una vez más, sólo necesitamos un proceso por adelantado para combinar los datos de los dos días para crear un full dataset y luego procesar eso. 

3. En algunos casos, podríamos recibir algunos archivos en un mismo día como full y otros como incrementales, y nuestro proceso tendrá que tratarlos. Por ejemplo, en el caso del dataset de Fórmula 1, nos encontraremos con esta situación. 

4. En algunos casos, podemos recibir datos incrementales e ingerirlos de forma incremental, pero las transformaciones se realizan en full Esto puede suceder porque el material de los datos que se reciben son incrementales y nuestra lógica de transformación puede requerir todos los datos. Por ejemplo, en nuestra Fórmula Uno, es posible que tengamos que agregar los datos de todas las carreras para averiguar los pilotos dominantes y los equipos dominantes y cosas por el estilo con los dos patrones de diseño primarios de carga incremental y la carga completa, así como estos escenarios híbridos. 